In [19]:
import sys
sys.path.append("/s/ls4/users/grartem/RL_robots/CommandClassifier")
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from collections import defaultdict
from copy import deepcopy
import torch

torch.cuda.is_available()
import yaml
import pyhocon
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import simpletransformers
from sklearn.metrics import classification_report
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs,
    ClassificationModel, ClassificationArgs
)
from RobotCommandClassifier.MyMultilabel import MyMultiLabelClassificationModel, MyMultiLabelClassificationArgs
from RobotCommandClassifier import utils
from RobotCommandClassifier import *

In [92]:
configFileContent = pyhocon.ConfigFactory.parse_file("../Configs/SimpleLM.conf")
CONFIG = configFileContent['rubert_tiny2_multilabel_noYno0_fold4'].as_plain_ordered_dict()
#configFileContent = pyhocon.ConfigFactory.parse_file("../Configs/CustomML.conf")
#CONFIG = configFileContent['MyMultiTiny2_att_fold0'].as_plain_ordered_dict()

In [93]:
train_x_df, train_y_df, valid_x_df, valid_y_df, test_x_df, test_y_df = utils.load_data(**CONFIG["Data"])

if CONFIG["Data"].get("add_y_to_x", False):
    with open(CONFIG["Data"]["y_descriptions_path"], "r") as f:
        y_descriptioons = json.load(f)
    train_x_df = train_x_df["y"].map(lambda y: y_descriptioons[int(y)]) + ": " + train_x_df["x"]
    valid_x_df = valid_x_df["y"].map(lambda y: y_descriptioons[int(y)]) + ": " + valid_x_df["x"]
    test_x_df = test_x_df["y"].map(lambda y: y_descriptioons[int(y)]) + ": " + test_x_df["x"]

if CONFIG["Data"].get("predict_label_flag", False):
    if "y" in CONFIG["Data"]["target_columns"]:
        raise ValueError("Указан флаг для использования только бинарных лейблов. Предполагается, что в таком случае 'y' не должен ыть среди target_columns.")
    train_y_df[train_y_df!=0] = 1
    valid_y_df[valid_y_df!=0] = 1
    test_y_df[test_y_df!=0] = 1
if CONFIG["Type"] in ["simple_ml_multilabel_classifier", "mymulti_classifier"]:
    if CONFIG["Data"].get("predict_label_flag", False):
        labels = train_y_df.values.tolist()
    else:
        enc = OneHotEncoder(**CONFIG["Data"].get("OneHotArgs", {}))
        enc.fit(train_y_df.values)

        labels = []
        encoded_labels = enc.transform(train_y_df.values).toarray().astype(int)
        for i in range(train_y_df.shape[0]):
            labels.append(encoded_labels[i].tolist())
    train_df = pd.DataFrame(list(zip(train_x_df, labels)))        
else:
    train_df = pd.concat([train_x_df, train_y_df], axis=1)
train_df.columns = ["text", "labels"]

if CONFIG["Type"] == "simple_ml_classifier":
    num_labels = len(train_y_df.iloc[:,0].unique())
else:
    num_labels = len(labels[0])

In [94]:
# Create model
CONFIG["Model"]["Args"]["output_dir"] = CONFIG["output_dir"]+"/models/"
CONFIG["Model"]["Args"]["best_model_dir"] = CONFIG["output_dir"] + "/models/best_model"
if CONFIG["Type"] == "simple_ml_multilabel_classifier":
    model_args = MultiLabelClassificationArgs(**CONFIG["Model"]["Args"])
    model = MultiLabelClassificationModel(
        CONFIG["Model"]["model_type"],
        CONFIG["output_dir"] + '/models/checkpoint-63780-epoch-10',
        num_labels=num_labels,
        use_cuda=True,    
        args=model_args,
    )
elif CONFIG["Type"] == "simple_ml_classifier":
    model_args = ClassificationArgs(**CONFIG["Model"]["Args"])
    model = ClassificationModel(
        CONFIG["Model"]["model_type"],
        CONFIG["output_dir"] + '/models/checkpoint-63780-epoch-10',
        num_labels=num_labels,
        use_cuda=True,    
        args=model_args,
    )
elif CONFIG["Type"] == "mymulti_classifier":
    model_args = MyMultiLabelClassificationArgs(**CONFIG["Model"]["Args"])
    # Create a MultiLabelClassificationModel
    model = MyMultiLabelClassificationModel(
        CONFIG["Model"]["model_type"],
        CONFIG["output_dir"] + '/models/checkpoint-63780-epoch-10',
        num_labels=num_labels,
        use_cuda=True,
        num_sublabels_per_biglabel = CONFIG["Model"]["num_sublabels_per_biglabel"],
        add_attention_for_labels=CONFIG["Model"]["add_attention_for_labels"],
        args=model_args,
    )
else:
    raise ValueError("unknown Type of experiment:{}".format(CONFIG["Type"]))

# Make prediction

In [95]:
predictions, raw_outputs = model.predict(test_x_df.values.tolist())

  0%|          | 0/263 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

In [96]:
#привести бинарный мультилейбл к мультиклассовому
with open("../Data/Interim/active_labels_for_y.json", "r") as f:
    active_labels_for_y = json.load(f)
predictions_2 = np.zeros((len(predictions), len(enc.categories_)))
for i in range(len(predictions)):
    shift = 0
    for j in range(len(enc.categories_)):
        if "OneHotArgs" in CONFIG["Data"] and CONFIG["Data"]["OneHotArgs"].get('drop', None) == "first":
            label_logits = raw_outputs[i, shift:shift+len(enc.categories_[j])-1]
            if sum(label_logits>=0.5)==0:
                predictions_2[i,j] = 0
            else:
                predictions_2[i,j] = np.argmax(label_logits)+1
            shift += len(enc.categories_[j])-1
        else:
            predictions_2[i,j] = np.argmax(raw_outputs[i, shift:shift+len(enc.categories_[j])])
            shift += len(enc.categories_[j])
    assert shift==len(predictions[0])

In [97]:
predictions = np.array(predictions)

In [98]:
if not os.path.exists(os.path.join(CONFIG["output_dir"], "reports")):
    os.mkdir(os.path.join(CONFIG["output_dir"], "reports"))
#result = utils.calculate_metrics_2(test_y_df.iloc[:,1:], predictions_2[:,1:], display=True) # исключить Y из оценки
result = utils.calculate_metrics_2(test_y_df, predictions_2, display=True)
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-10_classes_report.json"), "w") as f:
    json.dump(result, f)

#result_avg = utils.calculate_metrics(test_y_df.iloc[:,1:], predictions_2[:,1:], config={
result_avg = utils.calculate_metrics(test_y_df, predictions_2, config={
    "report_metrics": CONFIG["Report"]["report_metrics"]
})
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-10_avg_report.json"), "w") as f:
    json.dump(result_avg, f)
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

action
              precision    recall  f1-score   support

         0.0       0.44      0.70      0.54        10
         2.0       0.40      0.50      0.44         8
         3.0       0.00      0.00      0.00        10
         6.0       0.99      0.99      0.99       235
         7.0       0.00      0.00      0.00         0
         9.0       0.00      0.00      0.00         0

    accuracy                           0.93       263
   macro avg       0.30      0.37      0.33       263
weighted avg       0.91      0.93      0.92       263

direction
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99       245
         1.0       1.00      0.20      0.33         5
         2.0       0.00      0.00      0.00         3
         3.0       0.00      0.00      0.00         0
         4.0       1.00      0.80      0.89        10
         7.0       0.00      0.00      0.00         0

    accuracy                           0.97       263
   mac

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

72.0 	 correct_samples_perc
93.0 	 [action]_acc
33.0 	 [action]_macrof1
97.0 	 [direction]_acc
37.0 	 [direction]_macrof1
100.0 	 [meters]_acc
100.0 	 [meters]_macrof1
100.0 	 [degshours]_acc
100.0 	 [degshours]_macrof1
92.0 	 [object1]_acc
80.0 	 [object1]_macrof1
99.0 	 [nearest]_acc
50.0 	 [nearest]_macrof1
95.0 	 [relation1]_acc
96.0 	 [relation1]_macrof1
91.0 	 [object2]_acc
89.0 	 [object2]_macrof1
95.0 	 [relation2]_acc
95.0 	 [relation2]_macrof1
89.0 	 [object3]_acc
88.0 	 [object3]_macrof1
100.0 	 [self]_acc
100.0 	 [self]_macrof1
98.0 	 [gaze]_acc
50.0 	 [gaze]_macrof1
76.0 	 avg_macro_f1
96.0 	 avg_acc


/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

In [166]:
# выбрать только возможные или максимально правдоподобные
possible_combinations = pd.read_csv("../Data/Interim/possible_combinations.csv")
possible_tuples = []
for ir, row in possible_combinations.loc[:, CONFIG["Data"]["target_columns"]].iterrows():
    possible_tuples.append(tuple(row.values))

In [167]:
# Rule
with open("../Data/Interim/active_labels_for_y.json", "r") as f:
    active_labels_for_y = json.load(f)
predictions_3 = []
for i in range(len(predictions)):
    shift = 0
    predicted_tuples = []
    for cat_i in range(len(enc.categories_)):        
        probs = raw_outputs[i, shift:shift+len(enc.categories_[cat_i])]
        chosen_probs = np.where(probs>0.7)[0]
        if len(chosen_probs)==0:
            #chosen_probs = [np.argmax(probs)]
            #print("Low confidence", CONFIG["Data"]["target_columns"][cat_i], np.round(probs, 3))
            chosen_probs = np.argsort(probs)[-3:]
        if cat_i==0:
            for cls in chosen_probs:
                predicted_tuples.append([[cls], probs[cls]])
        else:
            if len(chosen_probs)==1:
                for tp in predicted_tuples:
                    tp[0].append(chosen_probs[0])
                    tp[1] += probs[chosen_probs[0]]
            elif len(chosen_probs)>1:
                #print(i, CONFIG["Data"]["target_columns"][cat_i], "more than 1 prob", chosen_probs)
                new_tuples = []
                for tp in predicted_tuples:
                    tp_template = deepcopy(tp)
                    for cls_i, cls in enumerate(chosen_probs):
                        if cls_i==0:
                            tp[0].append(cls)
                            tp[1] += probs[cls]
                        else:
                            new_tp = deepcopy(tp_template)
                            new_tp[0].append(cls)
                            new_tp[1] += probs[cls]
                            new_tuples.append(new_tp)
                predicted_tuples.extend(new_tuples)
        shift += len(enc.categories_[cat_i])
    possible_predicted_tuples = []
    for tp in predicted_tuples:
        assert len(tp[0])==len(CONFIG["Data"]["target_columns"])
        if tuple(tp[0]) in possible_tuples:
            possible_predicted_tuples.append([tp[0], tp[1]])
    
    if len(possible_predicted_tuples) ==0:
        print("No possible tuples for", i)
        print(predicted_tuples)
        predictions_3.append(sorted(predicted_tuples, key=lambda x: x[1])[-1][0])
    else:
        predictions_3.append(sorted(possible_predicted_tuples, key=lambda x: x[1])[-1][0])
predictions_3 = np.array(predictions_3)

No possible tuples for 116
[[[8, 6, 0, 0, 0, 8, 0, 2, 4, 1, 6, 0, 0], 11.85870361328125], [[8, 6, 0, 0, 0, 8, 0, 2, 4, 1, 4, 0, 0], 11.9217529296875], [[8, 6, 0, 0, 0, 8, 0, 2, 4, 1, 1, 0, 0], 12.33251953125]]


In [168]:
if not os.path.exists(os.path.join(CONFIG["output_dir"], "reports")):
    os.mkdir(os.path.join(CONFIG["output_dir"], "reports"))
result = utils.calculate_metrics_2(test_y_df, predictions_3, display=True)
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-10_classes_report_rule.json"), "w") as f:
    json.dump(result, f)

result_avg = utils.calculate_metrics(test_y_df, predictions_3, config={
    "report_metrics": CONFIG["Report"]["report_metrics"]
})
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-10_avg_report_rule.json"), "w") as f:
    json.dump(result_avg, f)
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

y
              precision    recall  f1-score   support

           0       0.82      0.90      0.86        10
           1       1.00      0.89      0.94        18
           5       0.82      0.82      0.82        11
           7       0.92      0.92      0.92        26
           8       0.99      0.99      0.99       199
          10       0.00      0.00      0.00         0

    accuracy                           0.97       264
   macro avg       0.76      0.75      0.76       264
weighted avg       0.97      0.97      0.97       264

action
              precision    recall  f1-score   support

           0       0.70      1.00      0.82         7
           1       0.00      0.00      0.00         3
           2       0.94      0.88      0.91        17
           3       0.00      0.00      0.00         1
           6       1.00      0.99      0.99       236
           9       0.00      0.00      0.00         0

    accuracy                           0.97       264
   macro avg  

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

81.0 	 correct_samples_perc
97.0 	 [y]_acc
76.0 	 [y]_macrof1
97.0 	 [action]_acc
45.0 	 [action]_macrof1
98.0 	 [direction]_acc
70.0 	 [direction]_macrof1
100.0 	 [meters]_acc
100.0 	 [meters]_macrof1
100.0 	 [degshours]_acc
100.0 	 [degshours]_macrof1
96.0 	 [object1]_acc
67.0 	 [object1]_macrof1
100.0 	 [nearest]_acc
100.0 	 [nearest]_macrof1
98.0 	 [relation1]_acc
98.0 	 [relation1]_macrof1
95.0 	 [object2]_acc
95.0 	 [object2]_macrof1
98.0 	 [relation2]_acc
98.0 	 [relation2]_macrof1
90.0 	 [object3]_acc
85.0 	 [object3]_macrof1
100.0 	 [self]_acc
50.0 	 [self]_macrof1
100.0 	 [gaze]_acc
100.0 	 [gaze]_macrof1
83.0 	 avg_macro_f1
98.0 	 avg_acc


/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [30]:
# Сохранить ошибки
with open("../Data/Interim/labels_names.json", "r") as f:
    labels_names = json.load(f)
predict_df = pd.DataFrame(predictions_2.astype(np.int))
predict_df.columns = [c+"_pred" for c in test_y_df.columns]
predict_df.index = test_y_df.index
errors = pd.concat([test_x_df, test_y_df, predict_df], axis=1)
for c in errors.columns:
    if c in "x":
        continue
    if "_pred" in c:
        errors[c] = errors[c].map(lambda x: labels_names[c.replace("_pred", "")][x])
    else:
        errors[c] = errors[c].map(lambda x: labels_names[c][x])
errors = errors.loc[:, ["x"] + [x for c in test_y_df.columns if c!= "x" for x in [c,c+"_pred"]]]

fullDF = pd.read_csv(CONFIG["Data"]["path_to_df"])
fullDF = fullDF[fullDF["subset"]=="test"]
errors = pd.concat([errors, fullDF.loc[fullDF.index.isin(errors.index),["type", "fold"]]], axis=1)

errors.to_csv("../Docs/rubert_tiny2_errors_fold4_test.csv", sep=";", encoding="cp1251", index=False)